In [1]:
%pip install -U langchain-community
%pip install langchain_groq langchain_huggingface langchain pypdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
GROQ = userdata.get("GROQ_API_KEY")

In [2]:
EMBEDDING = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

i:\RAG_C\pythonProject\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
llm = ChatGroq(groq_api_key = "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",model_name="llama3-groq-70b-8192-tool-use-preview")

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
  """
  Use the following pieces of context to answer the question at the end. If you dont know the answer just say that you dont know it.
{context}
Question : {question}
  """
)

In [8]:
pdf = "CONSTITUTION.pdf"

def vector_embeddings(data=pdf, max_documents = 50):
  embeddings = EMBEDDING
  loader = PyPDFLoader(data)
  documents = loader.load()[:max_documents]

  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
  )

  texts = text_splitter.split_documents(documents)
  return FAISS.from_documents(texts, embeddings)


In [9]:
vectors = vector_embeddings()

In [12]:
type(vectors)

langchain_community.vectorstores.faiss.FAISS

In [13]:
vectors.save_local("faiss_index")

In [15]:
new_vectors = FAISS.load_local(
    "faiss_index", embeddings=EMBEDDING, allow_dangerous_deserialization=True
)

In [16]:
docs = new_vectors.similarity_search("qux")

In [17]:
docs[0]

Document(metadata={'source': 'CONSTITUTION.pdf', 'page': 13}, page_content='the Chairman and Deputy Chairman.\n187. Secretariat of State Legislature.\nConduct of Business\n188. Oath or affirmation by members.\n189. Voting in Houses, power of Houses to act notwithstanding \nvacancies and quorum.\nDisqualifications of Members\n190. Vacation of seats.')

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vectors.as_retriever(),
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prompt}
)

In [ ]:
def response(query):
  result = qa_chain({"query": query})
  return result['result']

query = input("Enter your prompt : ")
response(query)

Enter your prompt : What basic fundamental rights does a person have in our constitution


'Based on the provided context, the following are some of the basic fundamental rights conferred by the Constitution of India:\n\n1. Right to Equality: This includes equality before law, prohibition of discrimination on grounds of religion, race, caste, sex or place of birth, and equality of opportunity in matters of public employment.\n\n2. Right to Freedom: This includes protection of certain rights regarding freedom of speech, protection in respect of conviction for offences, protection of life and personal liberty, and protection against arrest and detention in certain cases.\n\n3. Right against Exploitation: This includes prohibition of traffic in human beings and forced labour, and prohibition of employment of children in factories, etc.\n\n4. Right to Freedom of Religion: This includes freedom of conscience and free profession, practice and propagation of religion, freedom to manage religious affairs, and freedom as to payment of taxes for promotion of any particular religion.\n